In [90]:
# match product and listings : done, 3284 matches found
# A single price listing may match at most one product (canon digital ixus 1000 hs - marron has 2 records for price 422.99)
# are many such cases)
# Precision – do you make many false matches( Tried my best , by making the join using fields title & product_name as tight as possible)
# Recall – how many correct matches did you make(3284)
# Appropriate data structure and algorithm choices ( Used dict ,pandas,series )

In [9]:
import pandas as pd
import sqlite3
import json_lines

In [25]:
def createDF(filename):
    fname = filename
    cnt =1
    with open(fname, 'rb') as f:
        for item in json_lines.reader(f):
            if cnt == 1 :
                ds = pd.Series(item)
                df =pd.DataFrame(ds)
                toDF = df.transpose()
                cnt += 1
            else :
                ds2 = pd.Series(item)
                df2 =pd.DataFrame(ds2)
                df2 = df2.transpose()
                toDF = pd.concat([toDF,df2])
    return(toDF)

In [30]:
prodDF = createDF('products.txt')
lstDF = createDF('listings.txt')
print("products row count : ")
print(prodDF.count())
print("listings row count : ")
print(lstDF.count())

products row count : 
announced-date    743
family            485
manufacturer      743
model             743
product_name      743
dtype: int64
listings row count : 
currency        20196
manufacturer    20196
price           20196
title           20196
dtype: int64


In [79]:
#Data Prepration- to help with joins
new_prodDF= prodDF.replace({'_': ' '}, regex=True)
new_prodDF['product_name'] = new_prodDF['product_name'].str.lower()
new_prodDF.head(10)

,announced-date,family,manufacturer,model,product_name
0,2010-01-06T19:00:00.000-05:00,Cyber-shot,Sony,DSC-W310,sony cyber-shot dsc-w310
0,2010-01-05T19:00:00.000-05:00,NaN,Samsung,TL240,samsung tl240
0,2011-02-08T19:00:00.000-05:00,Coolpix,Nikon,S6100,nikon-s6100
0,2009-08-12T20:00:00.000-04:00,NaN,Samsung,TL220,samsung tl220
0,2011-02-15T19:00:00.000-05:00,FinePix,Fujifilm,T205,fujifilm-t205
0,1998-04-19T20:00:00.000-04:00,NaN,Casio,QV-5000SX,casio qv-5000sx
0,2010-02-07T19:00:00.000-05:00,Digital IXUS,Canon,130 IS,canon digital ixus 130 is
0,1998-09-15T20:00:00.000-04:00,NaN,Leica,Digilux,leica digilux
0,1999-05-17T20:00:00.000-04:00,FinePix,Fujifilm,1500,fujifilm finepix 1500
0,2011-02-01T19:00:00.000-05:00,Cybershot,Sony,DSC-HX100v,sony-hx100v


In [85]:
#we need to avoid false positive match,hence exclude null fields that make product_name coulmn
# count of null values in family,manufacturer & model

print(new_prodDF['family'].isnull().sum()) # only family has null values

print(new_prodDF['manufacturer'].isnull().sum())

print(new_prodDF['model'].isnull().sum())


258
0
0


In [35]:
new_lstDF= lstDF
new_lstDF['title'] = new_lstDF['title'].str.lower()
new_lstDF.head(5)

,currency,manufacturer,price,title
0,CAD,Neewer Electronics Accessories,35.99,led flash macro ring light (48 x led) with 6 a...
0,CAD,Canon Canada,199.96,canon powershot sx130is 12.1 mp digital camera...
0,CAD,Canon Canada,209.00,canon powershot sx130is 12.1 mp digital camera...
0,CAD,Canon Canada,306.24,canon powershot d10 12.1 mp waterproof digital...
0,CAD,Canon Canada,420.33,canon powershot d10 12.1 mp waterproof digital...


In [89]:
#we join these two data frames


In [180]:
import sqlite3
conn = sqlite3.connect(':memory:')
new_lstDF.to_sql('new_lstDF', conn, index=False)
new_prodDF.to_sql('new_prodDF', conn, index=False)
query = """
SELECT [new_prodDF].[product_name],
       [new_prodDF].[announced-date],
       [new_prodDF].[family],
       [new_prodDF].[model],
       [new_lstDF].[title],
       [new_lstDF].[manufacturer],
       [new_lstDF].[currency],
       [new_lstDF].[price]
FROM   ([new_lstDF]
        LEFT JOIN [new_prodDF]
                    on ([new_lstDF].[title]) LIKE ([new_prodDF].[product_name]||"%")
)   WHERE product_name IS NOT NULL
group by 1,2,3,4,5,6,7,8
"""
matchedDF = pd.read_sql_query(query, conn)
conn.close()
matchedDF

#WHERE [new_prodDF].[family] IS NOT Null
#       AND [new_prodDF].[model] IS NOT NULL
#       AND [new_prodDF].[manufacturer] IS NOT NULL

,product_name,announced-date,family,model,title,manufacturer,currency,price
0,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is - digital camera - c...,Canon,GBP,271.46
1,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - bla...,Canon,GBP,159.99
2,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - bla...,Canon,GBP,79.99
3,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - bla...,Canon,GBP,97.25
4,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - gol...,Canon,GBP,109.99
5,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - gol...,Canon,GBP,328.81
6,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - gol...,Canon,GBP,75.00
7,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - red...,Canon,GBP,109.99
8,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - red...,Canon,GBP,124.99
9,canon digital ixus 100 is,2009-02-17T19:00:00.000-05:00,Digital IXUS,100 IS,canon digital ixus 100 is digital camera - red...,Canon,GBP,79.00


In [181]:
matchedDF.count()

product_name      4522
announced-date    4522
family            3213
model             4522
title             4522
manufacturer      4522
currency          4522
price             4522
dtype: int64

In [182]:
matchedDF.groupby(['product_name','title', 'price']).count()
# All the product name and title have a count of one, though there are multiple prices

announced-date  \
product_name               title                                              price                     
canon digital ixus 100 is  canon digital ixus 100 is - digital camera - co... 271.46                1   
                           canon digital ixus 100 is digital camera - blac... 159.99                1   
                                                                              79.99                 1   
                                                                              97.25                 1   
                           canon digital ixus 100 is digital camera - gold... 109.99                1   
                                                                              328.81                1   
                                                                              75.00                 1   
                           canon digital ixus 100 is digital camera - red ... 109.99                1   
                                                                              124.99                1   
                                                                              79.00                 1   
                           canon digital ixus 100 is digital camera - silv... 109.99                1   
                                                                              159.99                1   
                           canon digital ixus 100 is digitalkamera (12 meg... 220.00                1   
                           canon digital ixus 100 is digitalkamera (12 meg... 250.00                1   
                                                                              90.00                 1   
canon digital ixus 1000 hs canon digital ixus 1000 hs - brown                 317.99                1   
                           canon digital ixus 1000 hs - brown + 16 gb sdhc... 364.26                1   
                           canon digital ixus 1000 hs - brown + 8 gb sdhc ... 355.62                1   
                           canon digital ixus 1000 hs - brown + 8 gb sdhc ... 362.51                1   
                           canon digital ixus 1000 hs - marron                331.63                1   
                                                                              422.99                1   
                           canon digital ixus 1000 hs - marron + carte mém... 467.99                1   
                           canon digital ixus 1000 hs - marron + carte mém... 452.99                1   
                           canon digital ixus 1000 hs - marron + carte mém... 461.99                1   
                           canon digital ixus 1000 hs - pink                  317.99                1   
                                                                              332.00                1   
                                                                              352.80                1   
                           canon digital ixus 1000 hs - pink + 16 gb sdhc ... 387.70                1   
                           canon digital ixus 1000 hs - pink + 16 gb sdhc ... 467.14                1   
                           canon digital ixus 1000 hs - pink + 8 gb sdhc m... 382.38                1   
...                                                                                               ...   
sony cyber-shot dsc-wx1    sony cyber-shot dsc-wx1/b 10mp "exmor r" cmos d... 199.99                1   
                                                                              219.95                1   
sony cyber-shot dsc-wx5    sony cyber-shot dsc-wx5 - gold                     239.00                1   
                                                                              292.00                1   
                           sony cyber-shot dsc-wx5 - gold + compact camera... 379.39                1   
                           sony cyber-shot dsc-wx5 - gold + compact pix ca... 314.16             

In [183]:
#matchedDF =matchedDF.drop(['announced-date','family','model'],axis='columns')  #Manage fields to be exported

In [184]:
#grpdMatchedDF=matchedDF.groupby('product_name')

In [214]:
matchedDF =matchedDF[['product_name','title','manufacturer','currency','price']]
matchedDF.head(20)

,product_name,title,manufacturer,currency,price
0,canon digital ixus 100 is,canon digital ixus 100 is - digital camera - c...,Canon,GBP,271.46
1,canon digital ixus 100 is,canon digital ixus 100 is digital camera - bla...,Canon,GBP,159.99
2,canon digital ixus 100 is,canon digital ixus 100 is digital camera - bla...,Canon,GBP,79.99
3,canon digital ixus 100 is,canon digital ixus 100 is digital camera - bla...,Canon,GBP,97.25
4,canon digital ixus 100 is,canon digital ixus 100 is digital camera - gol...,Canon,GBP,109.99
5,canon digital ixus 100 is,canon digital ixus 100 is digital camera - gol...,Canon,GBP,328.81
6,canon digital ixus 100 is,canon digital ixus 100 is digital camera - gol...,Canon,GBP,75.00
7,canon digital ixus 100 is,canon digital ixus 100 is digital camera - red...,Canon,GBP,109.99
8,canon digital ixus 100 is,canon digital ixus 100 is digital camera - red...,Canon,GBP,124.99
9,canon digital ixus 100 is,canon digital ixus 100 is digital camera - red...,Canon,GBP,79.00


In [229]:
out = matchedDF.to_json(orient='records')[1:-1].replace('},{', '}\n{')
with open('matchResult.txt', 'w') as f:
    f.write(out)

out = df20.to_json(orient='records')[1:-1].replace('},{', '} {')
with open('file_name.txt', 'w') as f:
    f.write(out)

df20['listings'] = df20[df20.columns[1:]].apply(lambda x: ','.join(x.dropna()),axis=1)
df20 = df20[['product_name','listings']]
df20